 # Americal Sign Language Classification using MobileNet

### Importing neccesary Modules

In [ ]:
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plt
import scipy.misc
import scipy
from scipy import ndimage
import numpy as np
import argparse
import sys
import os,datetime
# import pydot
from IPython.display import SVG
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import ops
from tensorflow.keras.applications import MobileNetV2 
%matplotlib inline

### Loading the Image dataset

In [ ]:
def datagen(batch=32):
    datagen1 = ImageDataGenerator( brightness_range=[0.8,1.5], horizontal_flip=True,shear_range=0.2, zoom_range=0.2, fill_mode="nearest", rescale=1./255)

    datagen2 = ImageDataGenerator( brightness_range=[0.5,1.7], horizontal_flip=True,shear_range=0.3, zoom_range=0.2, width_shift_range=0.1, fill_mode="nearest", rescale=1./255)
    
    train_generator = datagen1.flow_from_directory('ASL\asl_alphabet_train',shuffle=True, target_size = (200, 200), batch_size = batch, class_mode = 'categorical'  ) 

    val_generator = datagen2.flow_from_directory('ASL\asl_alphabet_test', target_size = (200, 200), batch_size = batch, class_mode = 'categorical' )

    return {"train":train_generator,"test":val_generator }

In [ ]:
GeneratedData =  datagen(32)
train = GeneratedData["train"]
test = GeneratedData["test"]

In [ ]:
X_train_orig, Y_train_orig=train.next()
X_test_orig, Y_test_orig=test.next()

In [ ]:
# Plotting Image for verification purposes
plt.imshow(X_train_orig[1])
print(np.argmax(Y_train_orig[1])+1)

## Our Model:


In [ ]:
from tensorflow.keras.models import Model 
from tensorflow.keras import backend as K
from keras import optimizers

In [ ]:
# Creating MobileNet instance
mobilenet_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(200,200,3))

m=mobilenet_model.output
m=tf.keras.layers.GlobalAveragePooling2D()(m)

# Adding dense layers so that the model can learn more complex functions and classify for better results.
m=tf.keras.layers.Dense(1280,activation='relu')(m) 
m=tf.keras.layers.Dropout(0.5)(m)

# Dense layer 2
m=tf.keras.layers.Dense(640,activation='relu')(m) 
m=tf.keras.layers.Dropout(0.3)(m)

# Dense layer 3
m=tf.keras.layers.Dense(320,activation='relu')(m) 

# Final Output layer with softmax activation function
mobile_preds = tf.keras.layers.Dense(29,activation='softmax')(m) 

mobile_model=Model(mobilenet_model.input,outputs=mobile_preds)

mobile_model.compile(loss='categorical_crossentropy',optimizer='Adadelta',metrics=['accuracy'])

### Summary of our model:

In [ ]:
mobile_model.summary()

### Visualizing the above through a graphical plot

In [ ]:
tf.keras.utils.plot_model(mobile_model, to_file='mobile_model.png')

### Training created model on augmented dataset 

In [ ]:
history = mobile_model.fit_generator(train,epochs=15,validation_data=test)

### Analyzing model training phase:

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### Saving model

In [ ]:
mobile_model.save("Mobile.h5")

<hr>